In [5]:
import pandas as pd
import numpy  as np
import matplotlib.pyplot as plt
import seaborn as sns

In [6]:
df = pd.read_csv(r"C:\Users\wecome\Downloads\amazon_alexa_data.csv")

In [7]:
df.head()

,Unnamed: 0,rating,date,variation,verified_reviews,feedback
0,0,5,31-Jul-18,Charcoal Fabric,Love my Echo!,1
1,1,5,31-Jul-18,Charcoal Fabric,Loved it!,1
2,2,4,31-Jul-18,Walnut Finish,"Sometimes while playing a game, you can answer...",1
3,3,5,31-Jul-18,Charcoal Fabric,I have had a lot of fun with this thing. My 4 ...,1
4,4,5,31-Jul-18,Charcoal Fabric,Music,1


In [8]:
df1 = df.copy()

In [9]:
df1.dropna(inplace=True)

In [10]:
df1.shape

(3150, 6)

In [11]:
df1.drop(['Unnamed: 0','rating','date','variation'],axis=1,inplace=True)

In [12]:
df1.head()

,verified_reviews,feedback
0,Love my Echo!,1
1,Loved it!,1
2,"Sometimes while playing a game, you can answer...",1
3,I have had a lot of fun with this thing. My 4 ...,1
4,Music,1


In [13]:
df1['feedback'].nunique()

2

In [14]:
df1['feedback'].value_counts()

1    2893
0     257
Name: feedback, dtype: int64

In [15]:
df1.loc[df['feedback']=='Extremely Positive','Sentiment'] = 'Positive'
df1.loc[df['feedback']=='Extremely Negative','Sentiment'] = 'Negative'

In [16]:
df1['feedback'].value_counts()

1    2893
0     257
Name: feedback, dtype: int64

In [17]:
import nltk
import string
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer

In [18]:
def remove_punctuation(text):
    no_punct = "".join([c for c in text if c not in string.punctuation])
    return no_punct

In [19]:
df1['verified_reviews'] = df1['verified_reviews'].apply(lambda x: remove_punctuation(x))
df1['verified_reviews'].head()

0                                         Love my Echo
1                                             Loved it
2    Sometimes while playing a game you can answer ...
3    I have had a lot of fun with this thing My 4 y...
4                                                Music
Name: verified_reviews, dtype: object

In [20]:
tokenizer = RegexpTokenizer(r'\w+')
df1['verified_reviews'] = df1['verified_reviews'].apply(lambda x: tokenizer.tokenize(x.lower()))
df1['verified_reviews'].head()

0                                     [love, my, echo]
1                                          [loved, it]
2    [sometimes, while, playing, a, game, you, can,...
3    [i, have, had, a, lot, of, fun, with, this, th...
4                                              [music]
Name: verified_reviews, dtype: object

In [21]:
lemmatizer = WordNetLemmatizer()

def word_lemmatizer(text):
    lem_text = [lemmatizer.lemmatize(i) for i in text]
    return lem_text

In [22]:
df1['verified_reviews'] = df1['verified_reviews'].apply(lambda x: word_lemmatizer(x))
df1['verified_reviews'].head()

0                                     [love, my, echo]
1                                          [loved, it]
2    [sometimes, while, playing, a, game, you, can,...
3    [i, have, had, a, lot, of, fun, with, this, th...
4                                              [music]
Name: verified_reviews, dtype: object

In [23]:
nltk.download("wordnet")
nltk.download("omw-1.4")

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\wecome\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\wecome\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [24]:
stemmer = PorterStemmer()

def word_stemmer(text):
    stem_text = " ".join([stemmer.stem(i) for i in text])
    return stem_text

In [25]:
df1['verified_reviews'] = df1['verified_reviews'].apply(lambda x: word_stemmer(x))
df1['verified_reviews']

0                                            love my echo
1                                                 love it
2       sometim while play a game you can answer a que...
3       i have had a lot of fun with thi thing my 4 yr...
4                                                   music
                              ...                        
3145         perfect for kid adult and everyon in between
3146    listen to music search locat check time look u...
3147    i do love these thing i have them run my entir...
3148    onli complaint i have is that the sound qualit...
3149                                                 good
Name: verified_reviews, Length: 3150, dtype: object

In [26]:
x = df1['verified_reviews']
y = df1['feedback']

In [27]:
from sklearn.model_selection import train_test_split

In [28]:
X_train,X_test,y_train,y_test = train_test_split(x,y,test_size=0.3,random_state=101)

In [29]:
X_train.shape

(2205,)

In [30]:
y_train.shape

(2205,)

In [31]:
X_test.shape

(945,)

In [32]:
y_test.shape

(945,)

In [33]:
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer

In [34]:
from sklearn.pipeline import Pipeline

In [35]:
count_vectorizer = CountVectorizer(stop_words='english')

In [36]:
my_pipeline = Pipeline(steps=[('CountVectorizer', CountVectorizer()),
                        ('Model', SVC())])

In [37]:
my_pipeline.fit(X_train,y_train)

Pipeline(steps=[('CountVectorizer', CountVectorizer()), ('Model', SVC())])

In [38]:
pred1 = my_pipeline.predict(X_test)

In [39]:
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
print("classification_report",classification_report(pred1,y_test))
print("Confusion Matrix", confusion_matrix(pred1,y_test))
print("Accuracy_score", accuracy_score(pred1,y_test))

classification_report               precision    recall  f1-score   support

           0       0.07      1.00      0.13         5
           1       1.00      0.93      0.96       940

    accuracy                           0.93       945
   macro avg       0.54      0.97      0.55       945
weighted avg       1.00      0.93      0.96       945

Confusion Matrix [[  5   0]
 [ 65 875]]
Accuracy_score 0.9312169312169312


In [40]:
my_pipeline2 = Pipeline(steps=[('Countvectorizer', CountVectorizer()),
                               ('Model', MultinomialNB())])

In [41]:
my_pipeline2.fit(X_train,y_train)

Pipeline(steps=[('Countvectorizer', CountVectorizer()),
                ('Model', MultinomialNB())])

In [42]:
pred2 = my_pipeline2.predict(X_test)

In [43]:

print("classification_report",classification_report(pred2,y_test))
print("Confusion Matrix", confusion_matrix(pred2,y_test))
print("Accuracy_score", accuracy_score(pred2,y_test))

classification_report               precision    recall  f1-score   support

           0       0.23      0.64      0.34        25
           1       0.99      0.94      0.96       920

    accuracy                           0.93       945
   macro avg       0.61      0.79      0.65       945
weighted avg       0.97      0.93      0.95       945

Confusion Matrix [[ 16   9]
 [ 54 866]]
Accuracy_score 0.9333333333333333


In [44]:
my_pipeline3 = Pipeline(steps=[('Countvectorizer', CountVectorizer()),
                               ('Model', KNeighborsClassifier())])

In [45]:
my_pipeline.fit(X_train,y_train)

Pipeline(steps=[('CountVectorizer', CountVectorizer()), ('Model', SVC())])

In [46]:
pred3 = my_pipeline.predict(X_test)

In [47]:

print("classification_report",classification_report(pred3,y_test))
print("Confusion Matrix", confusion_matrix(pred3,y_test))
print("Accuracy_score", accuracy_score(pred3,y_test))

classification_report               precision    recall  f1-score   support

           0       0.07      1.00      0.13         5
           1       1.00      0.93      0.96       940

    accuracy                           0.93       945
   macro avg       0.54      0.97      0.55       945
weighted avg       1.00      0.93      0.96       945

Confusion Matrix [[  5   0]
 [ 65 875]]
Accuracy_score 0.9312169312169312


In [48]:
from sklearn.linear_model import LogisticRegression

In [49]:
my_pipeline4 = Pipeline(steps=[('CountVectorizer', CountVectorizer()),
                               ('Model', LogisticRegression())])

In [50]:
my_pipeline4.fit(X_train,y_train)

Pipeline(steps=[('CountVectorizer', CountVectorizer()),
                ('Model', LogisticRegression())])

In [51]:
pred4 = my_pipeline4.predict(X_test)

In [52]:
print("classification_report",classification_report(pred4,y_test))
print("Confusion Matrix", confusion_matrix(pred4,y_test))
print("Accuracy_score", accuracy_score(pred4,y_test))

classification_report               precision    recall  f1-score   support

           0       0.41      0.74      0.53        39
           1       0.99      0.95      0.97       906

    accuracy                           0.95       945
   macro avg       0.70      0.85      0.75       945
weighted avg       0.96      0.95      0.95       945

Confusion Matrix [[ 29  10]
 [ 41 865]]
Accuracy_score 0.946031746031746


In [54]:
from sklearn.ensemble import RandomForestClassifier

In [55]:
my_pipeline5 = Pipeline(steps=[('Countvectorizer', CountVectorizer()),
                               ('Model', RandomForestClassifier())])

In [56]:
my_pipeline5.fit(X_train,y_train)

Pipeline(steps=[('Countvectorizer', CountVectorizer()),
                ('Model', RandomForestClassifier())])

In [57]:
pred5 = my_pipeline5.predict(X_test)

In [59]:
import pickle

In [58]:
print("classification_report",classification_report(pred5,y_test))
print("Confusion Matrix", confusion_matrix(pred5,y_test))
print("Accuracy_score", accuracy_score(pred5,y_test))

classification_report               precision    recall  f1-score   support

           0       0.24      0.94      0.39        18
           1       1.00      0.94      0.97       927

    accuracy                           0.94       945
   macro avg       0.62      0.94      0.68       945
weighted avg       0.98      0.94      0.96       945

Confusion Matrix [[ 17   1]
 [ 53 874]]
Accuracy_score 0.9428571428571428


svc: 

In [53]:
import joblib

In [54]:
joblib.dump(my_pipeline4,'Best Model.pkl')

['Best Model.pkl']

In [55]:
test = joblib.load("Best Model.pkl")

In [57]:
test.predict(['Sound is terrible if u want good music too get a bose'])

array([1], dtype=int64)